<a href="https://colab.research.google.com/github/kitcoolda/TS-Python-Misc/blob/main/DLF_B1_20210508.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Figures to words

In [ ]:
# Indian system data
in_denoms = {10000000: "crore", 100000: "lakh", 1000: "thousand", 100: "hundred", 1:""}
in_limit = 100 * 10000000

In [ ]:
# Western system data
us_denoms = {10**12 : "trillion", 10**9: "billion", 10**6: "million", 1000:"thousand", 1:""}
us_limit = 10 ** 15

In [ ]:
# Common data
SPACE = " "
HUN = "hundred"
upto20 = ["", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten", "eleven",
              "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "ninteen"]
tens = ["","", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

In [ ]:
def convert2digits(a: int) -> str:
    if a < 20:
        return upto20[a]
    return tens[a // 10] + SPACE + upto20[a % 10]

In [ ]:
convert2digits(54)

In [ ]:
convert2digits(17)

In [ ]:
def convert3digits(a: int) -> str:
    h, tu = a // 100, a % 100
    if h == 0:
        return convert2digits(tu)
    else:
        if tu == 0:
            glue = SPACE + HUN
        else:
            glue = SPACE + HUN + SPACE + "and" + SPACE
        return convert2digits(h) + glue + convert2digits(tu)

In [ ]:
convert3digits(312)

In [ ]:
convert3digits(79)

In [ ]:
convert3digits(400)

In [ ]:
def split_into_denoms(a, denoms):
    pieces = []
    for denom in denoms:
        pieces.append((a // denom, denom))
        a %= denom
    return pieces

In [ ]:
split_into_denoms(87167834, in_denoms)

In [ ]:
split_into_denoms(6697835674, us_denoms)

In [ ]:
def drop_zeros(pieces: [(int, int)]) -> [(int, int)]:
    return [x for x in pieces if x[0] != 0]

In [ ]:
drop_zeros(split_into_denoms(6697835674, us_denoms))

In [ ]:
def convert_pieces(pieces, denoms) -> [str]:
    return [convert3digits(x[0]) + SPACE + denoms[x[1]] for x in pieces]

In [ ]:
convert_pieces(drop_zeros(split_into_denoms(6697835674, us_denoms)), us_denoms)

In [ ]:
convert_pieces(drop_zeros(split_into_denoms(697835674, in_denoms)), in_denoms)

In [ ]:
def fix_and(s: str) -> str:
    if HUN not in s:
        return s
    if s.endswith(HUN):
        return s
    return s.replace(HUN, HUN + SPACE + "and")

In [ ]:
def fig2words(a: int, method: str):
    if method.upper().startswith("I"):
        denoms = in_denoms
        limit = in_limit
    else:
        denoms = us_denoms
        limit = us_limit
    if a >= limit:
        return f"{a} is too large. Must be less than {limit}"
    pieces = split_into_denoms(a, denoms)
    pieces = drop_zeros(pieces)
    words = convert_pieces(pieces, denoms)
    words = SPACE.join(words).capitalize()
    if limit == in_limit:
        return fix_and(words)
    else:
        return words

In [ ]:
fig2words(189756, "IN")

In [ ]:
fig2words(18975600, "IN")

In [ ]:
fig2words(78654345, "W")

In [ ]:
fig2words(1236576345, "IN")

# Odometer

In [ ]:
def is_ascending(n: int) -> bool:
    if n < 10:
        return True
    a, b = n // 10, n % 10
    if a % 10 >= b:
        return False
    return is_ascending(a)

In [ ]:
is_ascending(1234)

In [ ]:
is_ascending(1233)

In [ ]:
DIGITS = "123456789"
def limits(a: int) -> (int, int):
    size = len(str(a))
    return int(DIGITS[:size]), int(DIGITS[-size:])

In [ ]:
limits(234)

In [ ]:
limits(4444)

In [ ]:
def next_reading(r: int) -> int:
    lo, hi = limits(r)
    if r == hi:
        return lo
    r += 1
    while not (is_ascending(r)):
        r += 1
    return r

In [ ]:
next_reading(239)

In [ ]:
next_reading(next_reading(next_reading(679)))

In [ ]:
def prev_reading(r: int) -> int:
    lo, hi = limits(r)
    if r == lo:
        return hi
    r -= 1
    while not (is_ascending(r)):
        r -= 1
    return r

In [ ]:
def next_by(r: int, step:int) -> int:
    for _ in range(step):
        r = next_reading(r)
    return r

In [ ]:
next_by(679, 4)

In [ ]:
def prev_by(r: int, step:int) -> int:
    for _ in range(step):
        r = prev_reading(r)
    return r

In [ ]:
def diff(r1: int, r2: int):
    turns = 0
    while r1 != r2:
        turns += 1
        r1 = next_reading(r1)
    return turns

# OO Odometer

In [ ]:
class Odometer:
    DIGITS = "123456789"
    def __init__(self, size):
        self.SIZE = size
        self.LO = int(DIGITS[:size])
        self.HI = int(DIGITS[-size:])
        self.reading = self.LO
    
    def __str__(self):
        return str(self.reading)
    
    def __repr__(self):
        return f"{self.LO} <{self.reading}> {self.HI}"
    
    def is_valid(self):
        sr = str(self.reading)
        return all([a < b for a, b in zip(sr, sr[1:])])
    
    def forward(self, by = 1):
        for _ in range(by):
            if self.reading == self.HI:
                self.reading = self.LO
            else:
                self.reading += 1
                while not self.is_valid():
                    self.reading += 1

    def backward(self, by = 1):
        for _ in range(by):
            if self.reading == self.LO:
                self.reading = self.HI
            else:
                self.reading -= 1
                while not self.is_valid():
                    self.reading -= 1
    
    def diff(self, other):
        this = Odometer(self.SIZE)
        this.reading = self.reading
        turns = 0
        while this.reading != other.reading:
            this.forward()
            turns += 1
        return turns
        

In [ ]:
o = Odometer(4)

In [ ]:
o.forward(5)

In [ ]:
print(o)

In [ ]:
o